In [1]:
from kaggle_environments import make, evaluate

import main
import utils
from rl_agents.temporal_difference.tabular_agents import *
import arena

termcolor not installed, skipping dependency
No pygame installed, ignoring import
Loading environment lux_ai_2022 failed: No module named 'gym'


/Users/maxserra/OneDrive/Documents/MSc_Data_Science/ds_courses/ws2223_ml2/ml2_project_connectx/arena.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Check that env and render works...

In [2]:
env = make("connectx", configuration={"rows": 4, "columns": 4, "inarow": 3})

env.reset()
env.run(["random", "negamax"])
env.render(mode="ipython", width=400, height=350)

## Train an agent

In [5]:
env_configuration = {"rows": 6,
                     "columns": 7,
                     "inarow": 4}
env = make("connectx", configuration=env_configuration)

# q_agent = TabularQLearningAgent(actions=utils.get_possible_actions(n_cols=env_configuration["columns"]))

In [25]:
arena.train_agent(agent=q_agent,
                  env=env,
                  n_episodes=50)

Training 'TabularQLearningAgent' agent in 'connectx' environment...:   0%|          | 0/50 [00:00<?, ?it/s]

In [26]:
print(f"Number of explored states: {len(q_agent.q_values)}")
utils.dump_pretrained_agent(agent=q_agent,
                            file_name=utils.generate_agent_file_name(agent=q_agent, env=env))

Number of explored states: 219765
Dumping pretrained agent to 'pretrained_agents/TabularQLearningAgent_connectx_6x7row4.pkl'...


True

In [29]:
print(f"Number of explored states: {len(q_agent.q_values)}")

Number of explored states: 219765


Dumping pretrained agent to 'pretrained_agents/TabularQLearningAgent_connectx_6x7row4.pkl.xz'...


## Load and evaluate an agent

In [3]:
q_agent = utils.load_pretrained_agent(file_name="TabularQLearningAgent_connectx_6x7row4")
print(f"Number of explored states: {len(q_agent.q_values)}")

Loading pretrained agent from 'pretrained_agents/TabularQLearningAgent_connectx_6x7row4.pkl.xz'...
Number of explored states: 219765


In [6]:
def mean_reward(rewards, my_agent):
    return sum(r[my_agent] for r in rewards) / float(len(rewards))

n_episodes = 100
# Run multiple episodes to estimate its performance.
print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [q_agent.agent_func, "random"],
                                                        configuration=env_configuration, num_episodes=n_episodes), 0))
print("Random Agent vs My Agent:", mean_reward(evaluate("connectx", ["random", q_agent.agent_func],
                                                        configuration=env_configuration, num_episodes=n_episodes), 1))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [q_agent.agent_func, "negamax"],
                                                         configuration=env_configuration, num_episodes=n_episodes), 0))
print("Negamax Agent vs My Agent:", mean_reward(evaluate("connectx", ["negamax", q_agent.agent_func],
                                                         configuration=env_configuration, num_episodes=n_episodes), 1))

My Agent vs Random Agent: 0.36
Random Agent vs My Agent: -0.02
My Agent vs Negamax Agent: -0.94
Negamax Agent vs My Agent: -1.0


In [11]:
env.reset()
env.run([q_agent.agent_func, "negamax"])
env.render(mode="ipython", width=400, height=350)